In [1]:
import pandas as pd

df = pd.read_csv('/run/media/anton/Seagate Portable Drive/PNC/phenotypes.txt', sep='\t', comment='#')
print(df)

/tmp/ipykernel_3781592/2654265512.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/run/media/anton/Seagate Portable Drive/PNC/phenotypes.txt', sep='\t', comment='#')


       dbGaP_Subject_ID        SUBJID  INT_NUM INT_TYPE        Race Sex  \
0                791086  600001103037        1       MP          AA   M   
1                791086  600001103037        2       MI          AA   M   
2                791087  600001676724        1       AP          AA   F   
3                791088  600003245643        1      YPI          AA   M   
4                791090  600004963801        1       MP          EA   F   
...                 ...           ...      ...      ...         ...  ..   
14636            800198  609993878116        2       MI          EA   M   
14637            800199  609998081561        1       MP  EA,HI + OT   M   
14638            800199  609998081561        2       MI  EA,HI + OT   M   
14639            800200  609999144528        1       MP          EA   F   
14640            800200  609999144528        2       MI          EA   F   

       Med_birth_year Med_Rating  ADD011  ADD012  ...  SUB_OTC  SUB_PSY  \
0              1999.0   

In [2]:
import pickle

basedir = '/home/anton/Documents/Tulane/Research/PNC_Good/'
rest = pickle.load(open(f'{basedir}/PNC_SPM_rest.pkl', 'rb'))
nback = pickle.load(open(f'{basedir}/PNC_SPM_nback.pkl', 'rb'))
emoid = pickle.load(open(f'{basedir}/PNC_SPM_emoid.pkl', 'rb'))

print('Complete')

Complete


In [14]:
import numpy as np
from scipy import signal

def butter_bandpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = [cutoff[0] / nyq, cutoff[1] / nyq]
    b, a = signal.butter(order, normal_cutoff, btype='band', analog=False)
    return b, a

def butter_bandpass_filter(data, cutoff, fs, order=5):
    b, a = butter_bandpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

tr = 3

a,b = np.triu_indices(264,1)
y = []
ps = []

for i,ts in enumerate([rest, nback, emoid]):
    for sub in ts:
        val = list(df[df['SUBJID'] == int(sub)]['Race'].dropna())
        if len(val) == 0:
            continue
        if not all(v == val[0] for v in val):
            print(f'Bad {sub} {val}')
            continue
        if val[0] == 'EA':
            val = 1
        elif val[0] == 'AA':
            val = 0
        else:
            continue
#         val = int(val[0] == 'M')
        ts_filt = butter_bandpass_filter(ts[sub], [0.01, 0.15], 1/tr)
        p = np.corrcoef(ts_filt)
        if np.sum(np.isnan(p)) > 0:
            print(f'Bad ts {sub}')
            continue
        y.append(val)
        ps.append(p[a,b])
    print(f'Finished {i}')
    
y = np.stack(y)
ps = np.stack(ps)
    
print('Complete')
print(y[0:5])
print(ps.shape)

/usr/lib/python3.10/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/lib/python3.10/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Bad ts 601308609646
Bad ts 602818432787
Bad ts 605300352682
Bad ts 605515760919
Bad ts 606447154458
Bad ts 607218468837
Finished 0
Bad ts 602818432787
Finished 1
Finished 2
Complete
[0 1 1 1 1]
(3849, 34716)


In [26]:
from sklearn.cluster import KMeans

tgt = 1
y2 = np.zeros(y.shape)

km = KMeans(n_clusters=2, n_init='auto').fit(ps[y == tgt])
y2[y == tgt] = km.labels_ + 1

print(np.sum(y2 == 0), np.sum(y2 == 1), np.sum(y2 == 2))

# km = KMeans(n_clusters=2, n_init='auto').fit(ps)
# print(np.sum(km.labels_==0), np.sum(km.labels_==1))

1852 1257 740


In [26]:
ps2 = ps[km.labels_==0]
y2 = y[km.labels_==0]

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

idcs = np.arange(ps.shape[0]) 
# idcs = np.where(y2 != 1)[0]
xx = ps[idcs]
yy = y2[idcs]
yy = yy != 0
yy = yy.astype('int')

accs = []

for _ in range(20):
    # Select balanced set
    a, b = np.sum(yy == 0), np.sum(yy == 1)
    num = 779 #min(a,b)
    g1x, g2x = xx[yy == 0], xx[yy == 1]
    g1idcs = np.random.permutation(g1x.shape[0])[0:num]
    g2idcs = np.random.permutation(g2x.shape[0])[0:num]
    xxx = np.concatenate([g1x[g1idcs], g2x[g2idcs]])
    yyy = np.concatenate([np.zeros(num), np.ones(num)])
    
    xtr, xt, ytr, yt = train_test_split(xxx, yyy, stratify=yyy, train_size=0.8)

    print('Got to logistic')
    lr = LogisticRegression(C=100, max_iter=1000).fit(xtr, ytr)
    yhat = lr.predict(xt)
    acc = np.mean(yhat == yt)
    print(acc)
    
    accs.append(acc)
    
print('---')
print(np.mean(accs))
print(np.std(accs))

Got to logistic
0.8589743589743589
Got to logistic
0.8653846153846154
Got to logistic
0.8814102564102564
Got to logistic
0.8333333333333334
Got to logistic
0.8461538461538461
Got to logistic


KeyboardInterrupt: 

In [34]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

# def rmse(a,b):
#     return torch.mean((a-b)**2)**0.5

idcs = np.arange(ps.shape[0]) 
idcs = np.where(y2 != 1)[0]
xx = ps[idcs]
yy = y2[idcs]
yy = yy != 0
yy = yy.astype('int')
  
# xx = ps
# yy = y

accs = []

class Logistic(nn.Module):
    def __init__(self, dim):
        super(Logistic, self).__init__()
        self.fc = nn.Linear(dim,2).float().cuda()
        
    def reg_loss(self, C):
        return rmse(self.fc.weight, 0)
        
    def forward(self, x):
        return F.softmax(self.fc(x).squeeze(), dim=-1)
    
for _ in range(50):
    a, b = np.sum(yy == 0), np.sum(yy == 1)
    num = 740 #min(a,b)
    g1x, g2x = xx[yy == 0], xx[yy == 1]
    g1idcs = np.random.permutation(g1x.shape[0])[0:num]
    g2idcs = np.random.permutation(g2x.shape[0])[0:num]
    xxx = np.concatenate([g1x[g1idcs], g2x[g2idcs]])
    yyy = np.concatenate([np.zeros(num), np.ones(num)])

    xtr, xt, ytr, yt = train_test_split(xxx, yyy, stratify=yyy, train_size=0.8)
    xtr, xt, ytr = [torch.from_numpy(a).float().cuda() for a in [xtr, xt, ytr]]
    ytr = F.one_hot(ytr.long()).float()

    clf = Logistic(xtr.shape[-1])
    optim = torch.optim.Adam(clf.parameters(), lr=1e-4, weight_decay=1e-3)

    ce = nn.CrossEntropyLoss()

    for epoch in range(1000):
        optim.zero_grad()
        yhat = clf(xtr)
        loss = ce(yhat, ytr)
    #     reg_loss = clf.reg_loss(100)
        loss.backward()
        optim.step()
#         if epoch % 200 == 0:
#             print(f'{float(loss)}')

    yhat = clf(xt)
    yhat = torch.argmax(yhat, axis=-1)
    yhat = yhat.detach().cpu().numpy()
    acc = np.mean(yhat == yt)
    print(acc)
    accs.append(acc)
    
print('---')
print(np.mean(accs))
print(np.std(accs))

0.9324324324324325
0.8952702702702703
0.8885135135135135
0.902027027027027
0.8986486486486487
0.8952702702702703
0.875
0.8885135135135135
0.8885135135135135
0.8817567567567568
0.8716216216216216
0.918918918918919
0.8918918918918919
0.8851351351351351
0.8986486486486487
0.8918918918918919
0.9087837837837838
0.918918918918919
0.8783783783783784
0.8783783783783784
0.8952702702702703
0.8952702702702703
0.8885135135135135
0.8682432432432432
0.8851351351351351
0.8952702702702703
0.902027027027027
0.8682432432432432
0.8986486486486487
0.9222972972972973
0.8817567567567568
0.875
0.8851351351351351
0.9087837837837838
0.902027027027027
0.8952702702702703
0.8817567567567568
0.8817567567567568
0.8682432432432432
0.8783783783783784
0.8918918918918919
0.8885135135135135
0.8783783783783784
0.902027027027027
0.8817567567567568
0.8817567567567568
0.8851351351351351
0.9222972972972973
0.9054054054054054
0.8851351351351351
---
0.8917567567567568
0.014569548077776818
